In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib as plt
import numpy as np
from scipy.interpolate import interp1d

In [91]:
%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def get_pt_onclick(event):
    global ix,iy, rgba
    ix, iy = event.xdata,event.ydata
    rgba = event.inaxes.get_images()[0].get_cursor_data(event).data
    print(ix, iy)
    
def get_pts_manual(event):
    fig.canvas.draw_idle()
    global ix,iy, pts_manual
    if event.key == 'm':
        plt.plot(ix,iy, 'x', color = 'tab:red') 
        pts_manual.append([ix,iy])


def cal_points(event): 
    fig.canvas.draw_idle()
    global ix,iy, calpts
    if len(calpts)<4:
        #print(event.key)
        if event.key == 'enter':
            calpts.append([ix,iy])
            plt.plot(ix,iy, 'rx')
#             if len(calpts)>=4:
#                 plt.close(plt.gcf())


def bound_points(event): 
    fig.canvas.draw_idle()
    global ix,iy, boundpts
    if len(boundpts)<4:
        #print(event.key)
        if event.key == 'b':
            boundpts.append([ix,iy])
            plt.plot(ix,iy, 'gx')

                
def line_color(event):
    global ix,iy,rgba,line_col
    if event.key == '-':
        print(event.key)
        line_col = rgba 
        
def mouse_eraser(event):
    fig.canvas.draw_idle()
    global ix,iy, pts
    if event.key == ' ':
        plt.plot(event.xdata,event.ydata, 'bo') 
        try:
            idx = np.where(((pts[0,:] - event.xdata)**2 + (pts[1,:] - event.ydata)**2)<400)[0]
            pts[:, idx] = 0
        except:
            print('bleh')

In [114]:
ls

5ps_0.3mW_250kHz.npy    IMG_1829.png            IMG_1843.png
5ps_0.3mW_250kHz_2.npy  IMG_1830.HEIC           IMG_1844.png
5ps_18mW_250kHz.npy     IMG_1830.png            IMG_1845.png
5ps_3.3mW_250kHz.npy    IMG_1831.png            IMG_1846.png
5ps_31mW_250kHz.npy     IMG_1832.png            IMG_1847.png
5ps_45mW_250kHz.npy     IMG_1833.png            IMG_1848 2.png
5ps_58mW_250kHz.npy     IMG_1835.png            IMG_1851.png
5ps_70mW_250kHz.npy     IMG_1836.png            IMG_1852.png
5ps_8.5mW_250kHz.npy    IMG_1837.png            IMG_1853.png
5ps_8.5mW_250kHz_2.npy  IMG_1838.png            IMG_1854.png
5ps_84mW_250kHz.npy     IMG_1839.png            IMG_1855.png
Digitizer.ipynb         IMG_1840.png            IMG_1856.png
Digitizer_clean.ipynb   IMG_1841.png            README.md
IMG_1828.png            IMG_1842.png


In [115]:
filename = 'IMG_1842.png'

In [19]:
#set calpts (enter), boundpts (b), line_col (-)

In [116]:
img=mpimg.imread(filename)
fig = plt.figure(figsize=(10,5))
cid = fig.canvas.mpl_connect('button_press_event', get_pt_onclick)
kid = fig.canvas.mpl_connect('key_press_event', cal_points)
bid = fig.canvas.mpl_connect('key_press_event', bound_points)
lid = fig.canvas.mpl_connect('key_press_event', line_color)
eid = fig.canvas.mpl_connect('key_press_event', mouse_eraser)
mid = fig.canvas.mpl_connect('key_press_event', get_pts_manual)

ix,iy = 0,0
line_col = [0,0,0,0]
calpts = []
boundpts = []
pts_manual = []
imgplot = plt.imshow(img)
plt.show()

In [117]:
save_name = '5ps_103mW_250kHz'

1830.9181818181817 753.5363636363636
2284.7129996120457 1733.042048352157
2286.9948088781366 1730.760239086066
3551.1171422925954 1739.8874761504303
1928.7507541018367 1723.9148112877926
1933.314372634019 802.0638677869958
2880.265218061818 1393.052467704586
3035.4617669728364 1562.9135843042518
-


In [119]:
mask = np.sum((img - np.tile(line_col,(img.shape[0],img.shape[1],1)))**2, axis = 2)
mask2 = np.copy(mask)
mask2[mask2>0.02] = 0
X,Y=np.meshgrid(np.arange(mask2.shape[1]),np.arange(mask2.shape[0]))
bb = np.array(boundpts)
x0 = np.min(bb[:, 0]) #x
y0 = np.max(bb[:, 1]) #y
x1 = np.max(bb[:, 0]) #x
y1 = np.min(bb[:, 1]) #y
roi = np.zeros(mask2.shape)
roi[np.where((x0<X) & (y0>Y) & (x1>X) & (y1<Y))[0],np.where((x0<X) & (y0>Y) & (x1>X) & (y1<Y))[1]] = 1
plt.figure(1)
pts = np.array([np.where(mask2*roi>0)[1],np.where(mask2*roi>0)[0]])
plt.plot(pts[0,:], pts[1,:], 'x', color = 'palevioletred')
plt.show()

2166.058917775309 1075.8809797179256
2720.538569435442 790.6548214565405
2757.3637128688015 829.594872733852
2917.434630771987 903.5842337897345
2920.6969482788604 883.1947493717774
2921.1047379672195 870.1454793442849
2929.6683214227614 859.5429474469472
2929.6683214227614 867.6987412141301
2931.707269864557 879.5246421765452
2932.115059552916 888.0882256320872
2930.891690487839 902.7686544130163
2934.1540079947117 912.1478172452765
2933.3384286179935 928.4594047796421
2932.5228492412753 940.6930954304164
2935.7851667481486 956.5968932764229
2934.9695873714304 967.6072148621197
2938.2319048783033 976.9863776943799
2936.1929564365078 990.4434374102316
2904.385360744495 904.3998131664528
2908.055467939727 872.5922174744398
2904.793150432854 881.5635906183409
2895.0061979122343 943.9554129372896
2894.598408223875 954.9657345229864
2896.637356665671 965.9760561086832
2895.0061979122343 972.5006911224294
2895.0061979122343 981.0642745779714
2892.1516700937204 993.2979652287456
2891.3360907

In [120]:
#manual correction, erase (space), add (m)
if len(pts_manual) > 0: pts = np.hstack((pts,np.array(pts_manual).T))
pts = np.vstack((pts[0,:][(pts[0,:]>x0)&(pts[1,:]<y0)],pts[1,:][(pts[0,:]>x0)&(pts[1,:]<y0)]))
plt.figure(1)
plt.plot(pts[0,:], pts[1,:], 'gx')
plt.show()

2300.685664474683 1164.8715410954778


In [112]:
#interpolate and save
calval = [-40,-20,0.0,0.22]
fx = interp1d([calpts[0][0],calpts[1][0]], [calval[0],calval[1]],bounds_error=False, fill_value = 'extrapolate')
fy = interp1d([calpts[2][1],calpts[3][1]],[calval[2],calval[3]],bounds_error=False, fill_value = 'extrapolate')
data = {}
data['x'] = fx(pts[0,:])
data['y'] = fy(pts[1,:])
np.save(save_name+'.npy',data)

In [113]:
plt.figure(3)
plt.plot(data['x'],data['y'], 'o')